In [3]:
import numpy as np
import pandas as pd

data = pd.read_csv("data/movie.csv")
data.head()

,Unnamed: 0,name,genres,budget,revenue,runtime,popularity,vote_counts,vote_averages
0,0,Toy Story,"['Animation', 'Comedy', 'Family']",30000000.0,373554033.0,81.0,21.946943,5415.0,7.7
1,1,Jumanji,"['Adventure', 'Fantasy', 'Family']",65000000.0,262797249.0,104.0,17.015539,2413.0,6.9
2,2,Grumpier Old Men,"['Romance', 'Comedy']",0.0,0.0,101.0,11.712900,92.0,6.5
3,3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",16000000.0,81452156.0,127.0,3.859495,34.0,6.1
4,4,Father of the Bride Part II,['Comedy'],0.0,76578911.0,106.0,8.387519,173.0,5.7
